In [4588]:
import time
start_time = time.time()
import os
import psutil
from collections import OrderedDict
import operator
def readData():
    cnt = 0
    fp = open("data2.csv","r")   
    #fp = open("test.csv","r")
    dict_data = OrderedDict() # defaultdict(dict) 
    dict_count = {}
    for line in fp:
        if cnt == 0:
            val = line.strip('\n').split(";")
            val1 = val
        if cnt >= 1:
            val = line.strip('\n').split(";")
            dict_data[cnt-1] = val   #{val[i]: 1}
            for i in val:
                if i not in dict_count.keys():
                    dict_count[i] = 1
                else:
                    x = dict_count[i]
                    dict_count[i] = x + 1
        cnt += 1
    fp.close()
    tup_count = sorted(dict_count.items(), key=operator.itemgetter(1), reverse = True)
    dict_count = dict(tup_count)
    return dict_data,dict_count;

In [4589]:
 def create_header_table(data, d_count):
    header = d_count
    for k in header:
        header[k] = [header[k], None]
    return header
        

In [4590]:
def sort_data(dataset,header):
    sort_dict = {}
    frequent_itemset = set(header.keys())
    for itemset,count in dataset.items():
        frequent_transaction = {}
        for item in count:
            if item in frequent_itemset:
                frequent_transaction[item] = header[item][0]
            ordered_itemset = [v[0] for v in sorted(frequent_transaction.items(), key=lambda p: p[1], reverse=True)]
        sort_dict[itemset] = ordered_itemset
    return sort_dict

In [4591]:
class Tree_Node:
    def __init__(self,Nodename,parentnode,count):
        self.name = Nodename
        self.count = count
        self.parent = parentnode
        self.children = {}
        self.nodelink = None

In [4592]:
def create_tree(data,header,root):
    for val in data:
        if val in root.children:
            if root.name != 'root':
                root.children[val].count = root.children[val].count + 1#root.count = 0
            else:
                root.children[val].count = root.children[val].count + 1#root.count = root.count + 1
            root = root.children[val]
        else:
            root.children[val] = Tree_Node(val,root,1)
            root = root.children[val]
            if(header[val][1] == None):
                header[val][1] = root
            else:
                root_node = header[val][1]
                while(root_node.nodelink != None):
                    root_node = root_node.nodelink
                root_node.nodelink = root
            

In [4593]:
def printTree(theRoot):
    if(len(theRoot.children)==0):
        return
    for child in theRoot.children:
        printTree(theRoot.children[child])
        print("node is ", theRoot.children[child].name, "and count is ", theRoot.children[child].count)  
    

In [4594]:
def recur_link(node):
    print(node.name)
    if(node.nodelink==None):
        return
    recur_link(node.nodelink)
    
    

In [4595]:
def recur_getParentList(theNode):
    parentList=[]
    if(theNode.parent.name=="root"):
        parentList.append(theNode)
        return parentList
    parentList = parentList + recur_getParentList(theNode.parent)
    parentList.append(theNode)
    return parentList

In [4596]:
def create_sub_header(parentLists, theNodeList):
    index=0
    header={}
    for someList in parentLists:
        for someNode in someList:
            if someNode.name != "root":
                if someNode.name not in header.keys():
                    header[someNode.name]=theNodeList[index].count
                else:
                    header[someNode.name]=header[someNode.name]+theNodeList[index].count
        index=index+1
    for k in header:
        header[k] = [header[k], None]
    return header       

In [4597]:
def create_subtree(header, theRoot, parentList, theCount):
    for someNode in parentList:
        if someNode.name in theRoot.children:
            theRoot.children[someNode.name].count = theRoot.children[someNode.name].count + theCount
        else:
            theRoot.children[someNode.name] = Tree_Node(someNode.name, theRoot, theCount)
            if(header[someNode.name][1] == None):
                header[someNode.name][1] = theRoot.children[someNode.name]#someNode
            else:
                root_node = header[someNode.name][1]
                while(root_node.nodelink != None):
                    root_node = root_node.nodelink
                root_node.nodelink = theRoot.children[someNode.name]
        theRoot = theRoot.children[someNode.name]
        

In [4598]:
def recur_moreThan1path(theRoot):#root can have many branches but if there is only 1 path in each branch, return false
    temp = False
    if len(theRoot.children)==0:
        return False
    for child in theRoot.children:
        if len(theRoot.children)>1:
            return True 
        else:
            temp = recur_moreThan1path(theRoot.children[child]) 
        if temp == True:
            return True
    return temp 

In [4599]:
def gen_base_fp(theRoot, theHeader):
    patternLists=[]
    for child in theRoot.children:
        patternLists = patternLists + all_subsets(theRoot.children[child], [])
    for item in theHeader:
        found = False
        for someList in patternLists:
            if len(someList) == 2:
                if someList[-1]==item:
                    someList[0]=theHeader[item][0]
                    found = True
                    break
        if found == False:
            missingList = [theHeader[item][0], item]
            if missingList[0] >= 2:                                      #pruning
                patternLists.append(missingList)
    return patternLists

In [4600]:
def copyList(theList):
    copy = []
    index=0
    while index < len(theList):
        copy.append(theList[index])
        index=index+1
    return copy

In [4601]:
def all_subsets(theRoot, current_subsets):
    subsets = current_subsets
    next_set = []
    if theRoot.name!="root":
        if len(subsets)!=0:
            next_set=copyList(subsets[-1])
        else:
            next_set = [0]
        next_set.append(theRoot.name)
        next_set[0]=theRoot.count
        subsets.append(next_set)
    if(len(theRoot.children)==0):
        return subsets
    for child in theRoot.children:
        return all_subsets(theRoot.children[child], subsets)
        

In [4602]:
def pruneTree(theRoot):
    if theRoot.name!='root':
        if theRoot.count < 2:
            theRoot.parent.children.pop(theRoot.name)
            return
    if(len(theRoot.children)==0):
        return
    for child in list(theRoot.children):
        pruneTree(theRoot.children[child])

In [4603]:
def generated_patterns_recur(theNode, skip1):
    #print("* ",theNode.name)
    theNodeList=[]
    theHeader={}
    while(theNode.nodelink!=None):
        theNodeList.append(theNode)
        theNode=theNode.nodelink
    theNodeList.append(theNode)
    #print("theNodeList is ", theNodeList)             #list of nodes from header slot complete
    parentLists=[]
    parentList=[]
    for node in theNodeList:
        if(node.parent.name!="root"):
            parentList=recur_getParentList(node.parent)
            parentLists.append(parentList)
    #print(parentLists)             #lists of parents for each node from header slot complete
    theHeader=create_sub_header(parentLists, theNodeList)
    #print(theHeader)               #new header from parent paths has been made (still missing links)
    newRoot = Tree_Node('root', None, theNode.count)
    index1=0
    for someList in parentLists:
        create_subtree(theHeader, newRoot, someList, theNodeList[index1].count)#theNode.count
        index1=index1+1
    pruneTree(newRoot)
    #printTree(newRoot)                             #subTree created
    multiplePaths = False
    for child in newRoot.children:
        multiplePaths = recur_moreThan1path(newRoot.children[child])
        if multiplePaths == True:
            break
    #print("multpaths is ", multiplePaths)         #determined if there are multiple paths
    fpList=[]
    if skip1 == False:
        for someNode in theNodeList:
            if someNode.parent.name == 'root':
                if someNode.count >= 2:
                    fpList.append([someNode.count])#nodes whose only parents were the root were not used
                                                   #for subtrees. Now they get appended to output
    if multiplePaths == False:
        fpList = fpList + gen_base_fp(newRoot, theHeader)
    else:
        for elem in theHeader:
            fpList = fpList + generated_patterns_recur(theHeader[elem][1], False)
    for someList in fpList:
        someList.append(theNode.name)
    return fpList  

In [4604]:
printCount=0
data,d_count = readData()
header_table = create_header_table(data, d_count)
sort_dict = sort_data(data,header_table)
root = Tree_Node('root',None,0)
print("----")
for val in sort_dict.values():
    create_tree(val,header_table,root)
print("----")
for item in header_table:
    thePatterns = generated_patterns_recur(header_table[item][1], True)
    print("Patterns: ", thePatterns)
    printCount = printCount + len(thePatterns)
#print("Execution Time(Seconds)",time.time() - start_time)
#print("# of patterns printed ", printCount)
process = psutil.Process(os.getpid())
#print("Memory in MB",process.memory_info().rss/1000000) 

#there are 3 places pruning takes place


----
----
Patterns:  []
Patterns:  [[4, 'I2', 'I1']]
Patterns:  [[5, 'I2', 'I4'], [4, 'I1', 'I4']]
Patterns:  [[5, 'I2', 'I3'], [2, 'I2', 'I1', 'I3'], [4, 'I1', 'I3']]
Patterns:  [[2, 'I2', 'I5'], [2, 'I2', 'I1', 'I5'], [2, 'I1', 'I5']]
